# 6. Recommendations Sample

## Introduction

The purpose of this notebook is to reduce the size of the recommendations dataset that we can work on without incurring in memory issues

The filtering is done via **random sampling**

In [2]:
import pandas as pd
import numpy as np
import pickle
from scipy import sparse
import gzip


c:\Users\riccardoricci\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
### All random states are assigned to 42 for reproducible results
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [4]:
%run "1.Recommendation_Loading.ipynb"

## Filtering For Users and Games

In [1]:
def users_at_least_k_recs(df, K, n_users):
    """
    Return a list of users who perform at least K recommendations. Also, filtering for n_users randomly selected
    Args::
        df: dataframe, recommendation dataset
        K: integer, representing the minimum number of recommendations for a user to be included in the final list
        n_users: integer, number of randomly selected users
    Return:
        users: a list of randomly selected users who perform at least K recommendations
    """
    series = df.groupby("user_id_categorical")["user_id_categorical"].count()>=K
    series1 = series[series]
    users = list(series1.index)
    return np.random.choice(users, size = n_users)

users_to_keep = users_at_least_k_recs(recommendations, 20, 10000)
print("Number of users",  '{0:,.0f}'.format(len(users_to_keep)))
print("First five users id", users_to_keep[:5])


NameError: name 'recommendations' is not defined

## Loading the User-Game matrix and Apply Filtering

In [6]:
def decompress_pickle(input_file):
    with gzip.open(input_file, 'rb') as f:
        data = pickle.load(f)
    return data

user_game_matrix = decompress_pickle('matrix/user_game_matrix.pkl.gz')
user_game_matrix


<12663134x37420 sparse matrix of type '<class 'numpy.int32'>'
	with 47967516 stored elements in Compressed Sparse Row format>

In [7]:
# Convert this array/matrix to Dictionary Of Keys format
# user_game_matrix = user_game_matrix.todok()
print("Number of Rows:", '{0:,.0f}'.format(user_game_matrix.shape[0]))
print("Numbeer of Columns:", '{0:,.0f}'.format(user_game_matrix.shape[1]))
print("Number of stored values:", '{0:,.0f}'.format(user_game_matrix.size))


Number of Rows: 12,663,134
Numbeer of Columns: 37,420
Number of stored values: 47,967,516


In [8]:
# FILTERING the MATRIX for USERS to KEEP
# Matrix with only the subset of users who did at least k recs
mask = np.isin(np.array(user_game_matrix[:,0].todense()).reshape(-1), users_to_keep)
user_game_matrix_k_rec = user_game_matrix[mask]
print("Number of Rows:", '{0:,.0f}'.format(user_game_matrix_k_rec.shape[0]))
print("Numbeer of Columns:", '{0:,.0f}'.format(user_game_matrix_k_rec.shape[1]))
print("Number of stored values:", '{0:,.0f}'.format(user_game_matrix_k_rec.size))
user_game_matrix_k_rec

Number of Rows: 4,916
Numbeer of Columns: 37,420
Number of stored values: 191,000


<4916x37420 sparse matrix of type '<class 'numpy.int32'>'
	with 191000 stored elements in Compressed Sparse Row format>

In [9]:
# INSERTING a GAME ID ROW to the user-game matrix
games_id = np.arange(0, user_game_matrix_k_rec.shape[1]).reshape(1,-1)
games_id = sparse.csc_matrix(games_id)
user_game_matrix_k_rec = sparse.vstack((games_id, user_game_matrix_k_rec))
user_game_matrix_k_rec = user_game_matrix_k_rec.tocsc()
user_game_matrix_k_rec.todense()


matrix([[       0,        1,        2, ...,    37417,    37418,    37419],
        [   10783,        0,        0, ...,        0,        0,        0],
        [   11762,        0,        0, ...,        0,        0,        0],
        ...,
        [12659873,        0,        0, ...,        0,        0,        0],
        [12660359,        0,        0, ...,        0,        0,        0],
        [12662371,        0,        0, ...,        0,        0,        0]],
       dtype=int32)

In [10]:
# dropping games with no recommendations
mask = list(np.array(np.sum(user_game_matrix_k_rec[1:,1:], axis=0)>=1).reshape(-1))
mask.insert(0,True)
user_game_matrix_k_rec = user_game_matrix_k_rec.T[mask].T
print("Number of Rows:", '{0:,.0f}'.format(user_game_matrix_k_rec.shape[0]))
print("Numbeer of Columns:", '{0:,.0f}'.format(user_game_matrix_k_rec.shape[1]))
print("Number of stored values:", '{0:,.0f}'.format(user_game_matrix_k_rec.size))
user_game_matrix_k_rec.todense()

Number of Rows: 4,917
Numbeer of Columns: 17,109
Number of stored values: 208,108


matrix([[       0,        1,        2, ...,    37399,    37401,    37414],
        [   10783,        0,        0, ...,        0,        0,        0],
        [   11762,        0,        0, ...,        0,        0,        0],
        ...,
        [12659873,        0,        0, ...,        0,        0,        0],
        [12660359,        0,        0, ...,        0,        0,        0],
        [12662371,        0,        0, ...,        0,        0,        0]],
       dtype=int32)

In [11]:
# GAMES MAPPING 
# keys: integer, starting from 1 which are the columns of the filtered matrix
# values: original app_id_categorical
all_games = np.array(user_game_matrix_k_rec[0,1:].todense()).reshape(-1)
games_mapping = {k:i for k,i in enumerate(all_games)}
swapped_games_dict = {v: k for k, v in games_mapping.items()}
# swapped_games_dict[3481]

In [12]:
# USERS mapping
users_to_keep = np.array(user_game_matrix_k_rec[1:,0].todense()).reshape(-1)
users_mapping = {k:i for k,i in enumerate(users_to_keep)}
swapped_users_dict = {v: k for k, v in users_mapping.items()}


In [13]:
def build_recommendations_sample(df):
    """
    Filtering the recommendations dataset for the random sampled users 
    """
    recommendations_sample = df[(df["user_id_categorical"].isin(users_to_keep))]
    return recommendations_sample

recommendations_sample = build_recommendations_sample(recommendations)
recommendations_sample.head()


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical
38408207,238010,0,0,2019-12-01,1,39.0,12039,38408207,10783,1742
40762148,1782210,0,0,2021-12-14,1,5.0,12039,40762148,10783,33450
13596940,453480,12,6,2022-02-13,1,5.4,12039,13596940,10783,7783
5429077,381210,0,0,2021-10-20,1,10.0,12039,5429077,10783,5720
40340451,420110,4,2,2016-06-16,1,6.0,12039,40340451,10783,6843
